In [186]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [187]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random
import copy

In [188]:
%cd /content/drive/MyDrive/Colab Notebooks/BigData_proj/model

df_old =  pd.read_csv('../data/data/dataset_example.csv')

/content/drive/MyDrive/Colab Notebooks/BigData_proj/model


In [189]:
df = pd.read_csv('../data/data/dataset_example.csv')
#date 형식 변환
#df['date'] = [i.replace('-','') for i in df['date']]
#all_list : tournament,home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [190]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [191]:
#df_new
df_new = df
#len
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']

In [192]:
#df_new, df_label 분리 : 모델의 인풋(df_new)과 아웃풋(df_label, home_score 과 away score)
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)
#df_label
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']

**df_label** : 왜 인덱스가 7948까지 있을까? 표시는 6547 rows라고 되긴 함

In [193]:
df_label

,home_score,away_score
0,1.0,2.0
1,0.0,1.0
2,0.0,1.0
3,2.0,1.0
4,3.0,0.0
...,...,...
7944,1.0,1.0
7945,0.0,2.0
7946,1.0,1.0
7947,5.0,1.0


In [194]:
x_train = df_new.to_numpy()
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)

**df_train**

In [195]:
df_train

,Unnamed: 0,home_team,away_team,tournament,neutral,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,home_B3_point,...,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_W1_value,away_W2_value,away_G1_value,tor_len,home_len,away_len
0,36110,1,57,274,1,246.0,393.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,17,6,6
1,36111,2,57,274,1,249.0,393.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,17,7,6
2,36112,3,10,275,0,295.0,714.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,8,7,5
3,36113,4,16,275,0,656.0,290.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,8,7,8
4,36114,5,1,274,0,342.0,246.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,17,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,44054,35,22,316,0,1354.0,1384.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,19,7,7
6543,44055,18,178,316,0,1473.0,1563.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,19,6,6
6544,44056,128,184,316,0,1553.0,1384.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,19,6,8
6545,44057,59,45,316,0,1187.0,1183.0,42,68,54,...,34467857,25966145,10105847,21598364,11550757,13135984,8365789,19,6,6


In [196]:
#x_train = df_new.to_numpy()
Norm = Normalizer()
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

**df_temp** : df_train은 x_train을 사용, df_temp는 x_train을 normalize한 값인 x_temp 사용

In [197]:
df_temp

,Unnamed: 0,home_team,away_team,tournament,neutral,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,home_B3_point,...,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_W1_value,away_W2_value,away_G1_value,tor_len,home_len,away_len
0,0.000252,6.987484e-09,3.982866e-07,0.000002,6.987484e-09,0.000002,0.000003,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.187872e-07,4.192490e-08,4.192490e-08
1,0.000252,1.397497e-08,3.982866e-07,0.000002,6.987484e-09,0.000002,0.000003,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.187872e-07,4.891239e-08,4.192490e-08
2,0.000252,2.096245e-08,6.987484e-08,0.000002,0.000000e+00,0.000002,0.000005,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,5.589987e-08,4.891239e-08,3.493742e-08
3,0.000252,2.794993e-08,1.117997e-07,0.000002,0.000000e+00,0.000005,0.000002,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,5.589987e-08,4.891239e-08,5.589987e-08
4,0.000252,3.493742e-08,6.987484e-09,0.000002,0.000000e+00,0.000002,0.000002,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.187872e-07,3.493742e-08,4.192490e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,0.000308,2.445619e-07,1.537246e-07,0.000002,0.000000e+00,0.000009,0.000010,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.327622e-07,4.891239e-08,4.891239e-08
6543,0.000308,1.257747e-07,1.243772e-06,0.000002,0.000000e+00,0.000010,0.000011,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.327622e-07,4.192490e-08,4.192490e-08
6544,0.000308,8.943979e-07,1.285697e-06,0.000002,0.000000e+00,0.000011,0.000010,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.327622e-07,4.192490e-08,5.589987e-08
6545,0.000308,4.122615e-07,3.144368e-07,0.000002,0.000000e+00,0.000008,0.000008,2.934743e-07,4.751489e-07,3.773241e-07,...,0.240844,0.181438,0.070614,0.150918,0.080711,0.091787,0.058456,1.327622e-07,4.192490e-08,4.192490e-08


In [198]:
df_corr = df_temp.corr()

df_label과 df_temp(normalize된 값)

In [199]:
y_es = df_label.to_numpy()
x_es = df_temp.to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [200]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [201]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [ ]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

In [ ]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [ ]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

In [ ]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed)
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [ ]:
#inp = str(input("Enter Home & Away team (like Home Away): "))
autofill = True
#inp = inp.split(' ')
inp = ['Jordan', 'Morocco']

# **mixed** : seed = np.delete를 주석처리하면 해결됨. 기존 코드 체크하기

In [ ]:
def create_params():
  seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
  #seed = np.delete(seed, 4, axis=1)
  params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
  }
  return params

In [ ]:

#예선 팀 조 편성 결과 입력
Group_A= ["Uruguay","Russia","Saudi Arabia","Egypt"]
Group_B= ["Portugal","Spain","Morocco","Iran"]
Group_C= ["France","Denmark","Australia","Peru"]
Group_D= ["Argentina","Croatia","Iceland","Nigeria"]
Group_E= ["Brazil","Switzerland","Costa Rica","Serbia"]
Group_F= ["South Korea","Mexico","Sweden","Germany"]
Group_G= ["Belgium","England","Panama","Tunisia"]
Group_H= ["Senegal","Japan","Poland","Colombia"]
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}


In [ ]:
def create_input(home_team, away_team):
  #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
  ind_list = [home_team, away_team]
  return ind_list#input_vector

In [ ]:
def select_winning_team(ans, qual): #probability_array
    pred_ans = copy.deepcopy(ans)
    #예선 : 소수점 x, 정수로 승패 판결
    if qual == True :
      #정수
      pred_ans[0] = round(pred_ans[0])
      pred_ans[1] = round(pred_ans[1])
      
    #본선
    else :
      #소수점 1자리
      pred_ans[0] = round(pred_ans[0], 1)
      pred_ans[1] = round(pred_ans[1], 1)
    
    if (pred_ans[0]>pred_ans[1]):
          out=0 # home이 우승
    elif (pred_ans[0]<pred_ans[1]):
          out=1 # away가 우승
    elif (pred_ans[0]==pred_ans[1]):
          out=2 #비김

    return out, pred_ans

In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    #seed의 목적이 뭐지?
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed)
    
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [ ]:

##Group stage Matches
print("========Qualifying Games=========")
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"[{grp_name} Matches]")
    #해당 그룹의 팀들
    probable_countries=Groups[grp_name]

    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        #team i: team1, team j : team2
        j=i+1
        team_1=probable_countries[i]

        team_wins=0
        
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            team_lst=[team_1,team_2]

            #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
            input=create_input(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            #ans : 소수점 ex 1.3 : 1.5
            ans = predict_score(input, **params)
            #qual : 예선이면 true, 본선이면 false
            qual = True
           
            win, prob_lst=select_winning_team(ans, qual)
            print("applied_score : ", prob_lst,", predicted_score: ", ans)
            
            #본인 팀 스코어
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]
            #상대 팀 스코어
            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

            try:
                print(f" {team_1} vs {team_2} \n  {team_lst[win]} wins 🎊🏆⚽🎖️\n")
                #team1 우승
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    #team1 1승 추가
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    #team2 1패 추가
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    #비긴 팀 없음
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    #team2 1승 추가
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    #team1 1패 추가
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    #비긴 팀 없음
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
              #비긴 경우
              print(f"{team_1} vs {team_2} \n  Match Draw ⚽⚽⚽\n") 
              team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
              team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
              
              #비김
              draw_dct[team_1] = draw_dct.get(team_1,0)+1
              draw_dct[team_2] = draw_dct.get(team_2,0)+1
              #승패 없음
              win_dct[team_1] = win_dct.get(team_1,0)
              lost_dct[team_1] = lost_dct.get(team_1,0)
              #승패 없음
              win_dct[team_2] = win_dct.get(team_2,0)
              lost_dct[team_2] = lost_dct.get(team_2,0)


            j=j+1
    
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results


4 없애기 : 어떤 요소 없앴었다. 왜 없앴더라? 왜 기존 코드에선 없애도 되고 우리 코드에선 안되지?

**random 으로 seed를 왜 추출하는가?**